# Analyze Received Long Multi Tone Calibration WAV File
## By Terry Bondy, VA3TYB

In [1]:
printf(strftime ("Last updated: %A %e %B %Y %Z", localtime (time())))
%plot --format svg
format free
pkg load signal

Last updated: Sunday 12 January 2020 UTC

In [2]:
inputFilename = "/mnt/lexar/SDRuno_20200107_005334Z_445900kHz_trimmed.wav"

inputFilename = /mnt/lexar/SDRuno_20200107_005334Z_445900kHz_trimmed.wav


In [3]:
audioinfo(inputFilename)

ans =

  scalar structure containing the fields:

    Filename = /mnt/lexar/SDRuno_20200107_005334Z_445900kHz_trimmed.wav
    CompressionMethod = 
    NumChannels = 2
    SampleRate = 62500
    TotalSamples = 1.40625e+07
    Duration = 225
    BitsPerSample = -1
    BitRate = -1
    Title = 
    Artist = 
    Comment = 



In [4]:
[SamplesIQRV, sampleRate] = audioread(inputFilename);
size(SamplesIQRV)
sampleRate

ans =

 1.40625e+07 2


sampleRate = 62500


In [5]:
SamplesRV=SamplesIQRV(:,1)+i*SamplesIQRV(:,2);
size(SamplesRV)

ans =

 1.40625e+07 1




In [6]:
# plot(abs(SamplesRV));

In [7]:
numPeriods = rows(SamplesRV)/sampleRate;
period = numPeriods;
TshortRV = [0:1/sampleRate:1](:);
TfullRV = [0:1/sampleRate:period](:);

In [8]:
RaisedCosineRV = (1 - cos(2*pi*TshortRV))/2;
size(RaisedCosineRV)

ans =

 62501 1




In [9]:
SigSec0RV = SamplesRV(1:rows(RaisedCosineRV)) .* RaisedCosineRV;
size(SigSec0RV)

ans =

 62501 1




In [10]:
# plot(abs(SigSec0RV));

In [11]:
FftSec0RV = abs(fftshift(fft(SigSec0RV, sampleRate*10 + 1)));
FreqSec0RV = [linspace(-sampleRate/2, sampleRate/2, rows(FftSec0RV))](:);

In [12]:
% plot(FreqSec0RV, FftSec0RV)
% grid on
% grid minor
% axis([100 800], "tic")

In [13]:
% plot(FreqSec0RV, FftSec0RV)
% grid on
% grid minor
% idx = find(FftSec0RV(:) == max(FftSec0RV))
% FftSec0RV(idx)
% FreqSec0RV(idx)
% axis([ans-4 ans+4], "tic")

In [14]:
% plot(FreqSec0RV, FftSec0RV)
% grid on
% grid minor
% axis([235-4 235+4], "tic")

In [15]:
find(FreqSec0RV(:) == 230)

ans = 314801


In [16]:
find(FreqSec0RV(:) == 245)

ans = 314951


In [17]:
idx = find(FftSec0RV(:) == max(FftSec0RV(314801:314951)))
FftSec0RV(idx)
FreqSec0RV(idx)

idx = 314855
ans = 141.949
ans = 235.4


In [18]:
FreqSec0RV(find(FftSec0RV(:) == max(FftSec0RV(314801:314951)))) - ...
FreqSec0RV(find(FftSec0RV(:) == max(FftSec0RV)))

ans = -521


In [19]:
721 - 521

ans = 200


In [23]:
fracSeconds = 1;
indexIncr = (rows(TshortRV)-1)/fracSeconds;
pilotFreq = 721;
Freqs = [];
# Use a most positive initial reference freq because probe freq starts low and increases
refFreq = sampleRate * 2;
for startIndex = 1:indexIncr:rows(SamplesRV)-rows(RaisedCosineRV)-1
  time = TfullRV(startIndex)
  # SigSec0RV = SamplesRV(1:rows(RaisedCosineRV)) .* RaisedCosineRV;
  SigLoopRV = SamplesRV(startIndex:startIndex+rows(RaisedCosineRV)-1) .* RaisedCosineRV;
  FftLoopRV = fftshift(fft(SigLoopRV, sampleRate*10 + 1));
  FftLoopAbsRV = abs(FftLoopRV);
  # max(FftLoopAbsRV)
  # FftLoopPolCV = cart2pol(real(FftLoopRV), imag(FftLoopRV)); # rows of [theta, r]
  # FftLoopPolCV(1,:)
  FreqLoopRV = linspace(-sampleRate/2, sampleRate/2, rows(FftLoopRV));
  freqMaxPeak = FreqLoopRV(find(FftLoopAbsRV(:) == max(FftLoopAbsRV)))
  [PeaksCV idxPeaksCV] = findpeaks(FftLoopAbsRV, "MinPeakHeight", 50);
  # size(PeaksCV)
  # size(idxPeaksCV)
  PeaksCV
  if (rows(idxPeaksCV) == 1)
    idxPeaksCV = [idxPeaksCV; idxPeaksCV];
  endif
  # size(idxPeaksCV)
  # FreqLoopRV(idxPeaksCV)
  FreqPeaksLoopRV = FreqLoopRV(idxPeaksCV)
  FreqDiff = abs(FreqPeaksLoopRV .- refFreq);
  minFreqDiff = min(FreqDiff)
  # FftLoopPolCV(idxPeaksCV, :)
  cart2pol(real(FftLoopRV(idxPeaksCV)), imag(FftLoopRV(idxPeaksCV))) # rows of [theta, r]
  
  refFreqIdx = find(FreqDiff(:) == minFreqDiff)
  # Compute the new reference frequency
  refFreq = FreqPeaksLoopRV(refFreqIdx)
endfor

time = 0
freqMaxPeak = 756.4
PeaksCV =

 141.949
 407.382


FreqPeaksLoopRV =

 235.4 756.4


minFreqDiff = 124244
ans =

 2.58261 141.949
 -2.87714 407.382


refFreqIdx = 2
refFreq = 756.4
time = 1
freqMaxPeak = 756
PeaksCV =

 141.821
 408.2


FreqPeaksLoopRV =

 235 756


minFreqDiff = 0.4
ans =

 -1.1171 141.821
 -0.226931 408.2


refFreqIdx = 2
refFreq = 756
time = 2
freqMaxPeak = 755.6
PeaksCV =

 138.099
 406.494


FreqPeaksLoopRV =

 234.7 755.6


minFreqDiff = 0.4
ans =

 -1.18239 138.099
 0.0755653 406.494


refFreqIdx = 2
refFreq = 755.6
time = 3
freqMaxPeak = 755.2
PeaksCV =

 147.17
 407.625


FreqPeaksLoopRV =

 241.2 755.2


minFreqDiff = 0.4
ans =

 -2.16406 147.17
 -2.33525 407.625


refFreqIdx = 2
refFreq = 755.2
time = 4
freqMaxPeak = 754.7
PeaksCV =

 147.666
 406.125


FreqPeaksLoopRV =

 240.7 754.7


minFreqDiff = 0.5
ans =

 -0.927856 147.666
 -1.03754 406.125


refFreqIdx = 2
refFreq = 754.7
time = 5
freqMaxPeak = 754.3
PeaksCV =

 144.091
 406.369


FreqPeaksL